In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
from jupyter_dash import JupyterDash

In [3]:
# ANALYSIS OF INPUTS

app = Dash(__name__)


app.layout = html.Div([
    html.H1("Dashboard F2A"),
    html.H4("Weather Analysis"),
    html.Label("Variable:"),
    #dcc.Checklist(
    dcc.Dropdown(
    id='Variable',
    options=[
        {'label': 'Temperature', 'value': 'TM'},
        {'label': 'Wind Speed', 'value': 'WS'},
        {'label': 'Wind Direction', 'value': 'WD'},
        {'label': 'Relative Humidity', 'value': 'RH'},
    ],
    value='TM'
    ),
    html.Br(),
    html.Label("Select aggregation by weather file:"),
    dcc.Dropdown(
    id='Aggregation',
    options=[
        {'label': 'No aggregation', 'value': 'No aggregation'},
        {'label': 'Max by file', 'value': 'Max by file'},
    ],
    value='No aggregation'
    ),
    dcc.Graph(id="graph"),
])

@app.callback(
    Output("graph", "figure"), 
    Input("Aggregation", "value"),
    Input("Variable", "value"))
def generate_chart(aggregation,variable):
    if aggregation == "Max by file":
        df = df_weathers.groupby(['Scenario Group', 'Scenario']).max().reset_index()
    else:
        df = df_weathers
    fig = px.box(df, x="Scenario Group", y=variable, title=f'Boxplot {variable}')
    return fig

app.run_server(jupyter_mode="tab")

Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>

# Dashboard

In [2]:
def ROS_Kitral(TM, RH, WS, fmc):

    elev_j = 0 
    elev_i = 0
    cell_size = 100

    se = 1 + 0.023322 * (elev_j - elev_i)/(cell_size/4) + 0.00013585 * ((elev_j - elev_i)/(cell_size/4))**2

    sigmoid_ch = 1/(1 + np.exp(-0.081*(RH - 57.09)))
    ch = 4 + 16*sigmoid_ch - 0.00982*TM 
    fch = np.minimum(51.43, 52.3342*ch**(-1.3035))

    fv = -12.86*np.exp(-0.04316*WS) + 13.8

    ros = fmc * fch * (fv + se)

    return ros

In [ ]:
df = pd.DataFrame(columns=['FMC', 'CBH', 'FL', 'H', 'CBD'])

Modelo para BN03:
    fmc_bn03 = 0.000979
    cbh_bn03 = 0
    fl_bn03 = 3.544 (fuel load)
    h_bn03 = 19045 (heat yield)
    cbd_bn03 = 0

Modelo para PL01:
    fmc_pl01= 0..013174
    cbh_pl01 = 0.93
    fl_pl01 = 0.838
    h_pl01 = 18405
    cbd_pl01 = 0.03

In [25]:
app = Dash(__name__)

app.layout = html.Div([
    html.H1("Dashboard F2A"),
    html.H2("Equation Analysis"),
    html.Label("Select the fuel model:"),
    dcc.Dropdown(
    id='Fuel_model',
    options=[
        {'label': 'Kitral', 'value': 'Kitral'},
    ],
    value='Kitral'
    ),

    # ROS VS TM
    html.Br(),
    html.H4('ROS vs Temperature'),
    html.Br(),
    html.Label('Select the Temperature (TM) Range:'),
    html.Br(),
    dcc.RangeSlider(0, 50, value=[15, 40], id="Temperature_Range",
                tooltip={"placement": "bottom", "always_visible": True}),
    html.Br(),
    html.Label("Select the Relative Humidity (RH):"),
    dcc.Slider(0, 100,
               value=17,
               tooltip={"placement": "bottom", "always_visible": True},
               id='RH'
    ),
    html.Br(),
    html.Label("Select the Wind Speed (WS):"),
    dcc.Slider(0, 100,
               value=30,
               tooltip={"placement": "bottom", "always_visible": True},
               id='WS'
    ),
    html.Br(),
    dcc.Graph(id="graph_TM"),
])

@app.callback(
    Output('graph_TM', 'figure'),
    Input('Fuel_model', 'value'),
    Input('Temperature_Range', 'value'),
    Input('RH', 'value'),
    Input('WS', 'value'),
    )
def update_output(fuel_model, tm_range, rh, ws):

    TM = np.linspace(tm_range[0],tm_range[1], 100)
    RH = rh
    WS = ws
    fmc = 0.013174 # depende del combustible

    if fuel_model == 'Kitral':
        ROS = ROS_Kitral(TM, RH, WS, fmc)
        fig = px.line(x=TM, y=ROS, title="ROS vs Temperature") # Falta titulos, ejes (nombres y coordenadas fijas), leyenda

    return fig

app.run_server(jupyter_mode="tab")


Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>

In [ ]:
app = Dash(__name__)

app.layout = html.Div([
    html.H1("Dashboard F2A"),
    html.H2("Equation Analysis"),
    html.Label("Select the fuel model:"),
    dcc.Dropdown(
    id='Fuel_model',
    options=[
        {'label': 'Kitral', 'value': 'Kitral'},
    ],
    value='Kitral'
    ),

    # ROS VS TM
    html.Br(),
    html.H4('ROS vs Relative Humidity'),
    html.Br(),
    html.Label('Select the Relative Humidity (RH) Range:'),
    html.Br(),
    dcc.RangeSlider(0, 100, value=[5, 30], id="RH_Range",
                tooltip={"placement": "bottom", "always_visible": True}),
    html.Br(),
    html.Label("Select the Temperature (TM):"),
    dcc.Slider(0, 50,
               value=30,
               tooltip={"placement": "bottom", "always_visible": True},
               id='TM'
    ),
    html.Br(),
    html.Label("Select Wind Speed (WS):"),
    dcc.Slider(0, 100,
               value=30,
               tooltip={"placement": "bottom", "always_visible": True},
               id='WS'
    ),
    html.Br(),
    dcc.Graph(id="graph_RH"),
])

@app.callback(
    Output('graph_RH', 'figure'),
    Input('Fuel_model', 'value'),
    Input('RH_Range', 'value'),
    Input('TM', 'value'),
    Input('WS', 'value'),
    )
def update_output(fuel_model, rh_range, tm, ws):

    RH = np.linspace(rh_range[0],rh_range[1], 100)
    TM = tm
    WS = ws
    fmc = 0.013174 # depende del combustible

    if fuel_model == 'Kitral':
        ROS = ROS_Kitral(TM, RH, WS, fmc)
        fig = px.line(x=RH, y=ROS, title="ROS vs Relative Humidity") # Falta titulos, ejes (nombres y coordenadas fijas), leyenda

    return fig

app.run_server(jupyter_mode="tab")

Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>

In [ ]:
app = Dash(__name__)

app.layout = html.Div([
    html.H1("Dashboard F2A"),
    html.H2("Equation Analysis"),
    html.Label("Select the fuel model:"),
    dcc.Dropdown(
    id='Fuel_model',
    options=[
        {'label': 'Kitral', 'value': 'Kitral'},
    ],
    value='Kitral'
    ),

    # ROS VS TM
    html.Br(),
    html.H4('ROS vs Temperature'),
    html.Br(),
    html.Label('Select the Temperature (TM) Range:'),
    html.Br(),
    dcc.RangeSlider(0, 50, value=[15, 40], id="Temperature_Range",
                tooltip={"placement": "bottom", "always_visible": True}),
    html.Br(),
    html.Label("Select the Relative Humidity (RH):"),
    dcc.Slider(0, 100,
               value=17,
               tooltip={"placement": "bottom", "always_visible": True},
               id='RH_1'
    ),
    html.Br(),
    html.Label("Select the Wind Speed (WS):"),
    dcc.Slider(0, 100,
               value=30,
               tooltip={"placement": "bottom", "always_visible": True},
               id='WS_1'
    ),
    html.Br(),
    dcc.Graph(id="graph_TM"),

    # ROS VS RH
    html.Br(),
    html.H4('ROS vs Relative Humidity'),
    html.Br(),
    html.Label('Select the Relative Humidity (RH) Range:'),
    html.Br(),
    dcc.RangeSlider(0, 100, value=[5, 30], id="RH_Range",
                tooltip={"placement": "bottom", "always_visible": True}),
    html.Br(),
    html.Label("Select the Temperature (TM):"),
    dcc.Slider(0, 50,
               value=30,
               tooltip={"placement": "bottom", "always_visible": True},
               id='TM_2'
    ),
    html.Br(),
    html.Label("Select Wind Speed (WS):"),
    dcc.Slider(0, 100,
               value=30,
               tooltip={"placement": "bottom", "always_visible": True},
               id='WS_2'
    ),
    html.Br(),
    dcc.Graph(id="graph_RH"),

    # ROS VS WS
    html.Br(),
    html.H4('ROS vs Relative Wind Speed'),
    html.Br(),
    html.Label('Select the Wind Speed (WS) Range:'),
    html.Br(),
    dcc.RangeSlider(0, 100, value=[5, 30], id="WS_Range",
                tooltip={"placement": "bottom", "always_visible": True}),
    html.Br(),
    html.Label("Select the Temperature (TM):"),
    dcc.Slider(0, 50,
               value=30,
               tooltip={"placement": "bottom", "always_visible": True},
               id='TM_3'
    ),
    html.Br(),
    html.Label("Select Relative Humidity (RH):"),
    dcc.Slider(0, 100,
               value=30,
               tooltip={"placement": "bottom", "always_visible": True},
               id='WS_3'
    ),
    html.Br(),
    dcc.Graph(id="graph_WS"),
])




@app.callback(
    Output('graph_TM', 'figure'),
    Input('Fuel_model', 'value'),
    Input('Temperature_Range', 'value'),
    Input('RH_1', 'value'),
    Input('WS_1', 'value'),
    )
def update_output_TM(fuel_model, tm_range, rh, ws):

    TM = np.linspace(tm_range[0],tm_range[1], 100)
    RH = rh
    WS = ws
    fmc = 0.013174 # depende del combustible

    if fuel_model == 'Kitral':
        ROS = ROS_Kitral(TM, RH, WS, fmc)
        fig = px.line(x=TM, y=ROS, title="ROS vs Temperature") # Falta titulos, ejes (nombres y coordenadas fijas), leyenda

    return fig


@app.callback(
    Output('graph_RH', 'figure'),
    Input('Fuel_model', 'value'),
    Input('RH_Range', 'value'),
    Input('TM_2', 'value'),
    Input('WS_2', 'value'),
    )
def update_output_RH(fuel_model, rh_range, tm, ws):

    RH = np.linspace(rh_range[0],rh_range[1], 100)
    TM = tm
    WS = ws
    fmc = 0.013174 # depende del combustible

    if fuel_model == 'Kitral':
        ROS = ROS_Kitral(TM, RH, WS, fmc)
        fig = px.line(x=RH, y=ROS, title="ROS vs Relative Humidity") # Falta titulos, ejes (nombres y coordenadas fijas), leyenda

    return fig

@app.callback(
    Output('graph_WS', 'figure'),
    Input('Fuel_model', 'value'),
    Input('WS_Range', 'value'),
    Input('TM_3', 'value'),
    Input('RH_3', 'value'),
    )
def update_output_WS(fuel_model, ws_range, tm, rh):

    WS = np.linspace(ws_range[0],ws_range[1], 100)
    TM = tm
    RH = rh
    fmc = 0.013174 # depende del combustible

    if fuel_model == 'Kitral':
        ROS = ROS_Kitral(TM, RH, WS, fmc)
        fig = px.line(x=WS, y=ROS, title="ROS vs Wind Speed") # Falta titulos, ejes (nombres y coordenadas fijas), leyenda

    return fig

app.run_server(jupyter_mode="tab")

Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>